In [1]:
import pandas as pd
import numpy as np

In [2]:
def hamilton_product(a: np.array, b: np.array):
    """
    Hamilton product for two quaternions or a Vec4 and a Quaternion.
    :param a: quaternion or vec4 in order [w,x,y,z]
    :param b: quaternion in order [w,x,y,z]
    """
    # check shape to deal with a whole column of rotations
    if len(a.shape) > 1:
        a = [a[:, 0], a[:, 1], a[:, 2], a[:, 3]]
    if len(b.shape) > 1:
        b = [b[:, 0], b[:, 1], b[:, 2], b[:, 3]]
    h_p = np.array([
        a[0] * b[0] - a[1] * b[1] - a[2] * b[2] - a[3] * b[3],
        a[0] * b[1] + a[1] * b[0] + a[2] * b[3] - a[3] * b[2],
        a[0] * b[2] - a[1] * b[3] + a[2] * b[0] + a[3] * b[1],
        a[0] * b[3] + a[1] * b[2] - a[2] * b[1] + a[3] * b[0]
    ], dtype=np.float64)
    if len(h_p.shape) > 1:
        return h_p.transpose()
    else:
        return h_p
    

def quat_invert(q: np.array):
    """
    estimates the inverse rotation.
    :param q: input quaternion
    :return: inverse quaternion
    """
    q_s = q * np.array([1, -1, -1, -1], dtype=np.float64)  # the conjugate of the quaternion
    if len(q.shape) > 1:
        return q_s / np.sum(np.square(q, dtype=np.float64), axis=1, keepdims=True, dtype=np.float64)
    else:
        return q_s / np.sum(np.square(q, dtype=np.float64), dtype=np.float64)

In [3]:
#Add file name you want to update
data = pd.read_csv("/Users/affanbinusman/Dropbox (ASU)/IRL-Lab/P&G/hackathon_data/")
print(data.shape)


(26579, 75)


In [4]:
sw_forward = np.array(data[["sw_forward_w", "sw_forward_x", "sw_forward_y", "sw_forward_z"]])
sw_rotvec = np.array(data[["sw_rotvec_w", "sw_rotvec_x", "sw_rotvec_y", "sw_rotvec_z"]])
hamilton = hamilton_product(quat_invert(sw_forward), sw_rotvec)

In [5]:
data["sw_rot_cal_w"] = hamilton[:, 0]
data["sw_rot_cal_x"] = hamilton[:, 1]
data["sw_rot_cal_y"] = hamilton[:, 2]
data["sw_rot_cal_z"] = hamilton[:, 3]

In [6]:
sw_forward = np.array(data[["ph_forward_w", "ph_forward_x", "ph_forward_y", "ph_forward_z"]])
sw_rotvec = np.array(data[["ph_rotvec_w", "ph_rotvec_x", "ph_rotvec_y", "ph_rotvec_z"]])
hamilton = hamilton_product(quat_invert(sw_forward), sw_rotvec)

In [7]:
data["ph_rot_cal_w"] = hamilton[:, 0]
data["ph_rot_cal_x"] = hamilton[:, 1]
data["ph_rot_cal_y"] = hamilton[:, 2]
data["ph_rot_cal_z"] = hamilton[:, 3]

In [8]:
data["pres_cal"] = data["sw_pres"] - data["sw_rel_pres"]

In [9]:
print(data.shape)

(26579, 84)


In [10]:
# Are you sure you want to overwrite? Are you sure sure?
# data.to_csv('/Users/affanbinusman/Dropbox (ASU)/IRL-Lab/P&G/hackathon_data/', index=False)